# Lesson 8: ReAct Practice

This notebook explores practical the ReAct (Reasoning and Acting) pattern with Google's Gemini API. We will use the `google-genai` library to interact with Gemini models. It includes a mock search tool, a thought generation phase using structured outputs, and an action phase with function calling, all orchestrated by a ReAct control loop.

Learning Objectives:

1. Understand how ReAct breaks problems into Thought → Action → Observation.
2. Practice orchestrating the full ReAct loop end-to-end.

## 1. Setup

### Set Up Python Environment

Run the following command to install all the required packages to run this notebook.

In [ ]:
%pip install -q \
  agentic-ai-engineering-course \
  google-auth==2.38.0 \
  opentelemetry-api==1.37.0 opentelemetry-sdk==1.37.0 \
  opentelemetry-exporter-otlp-proto-http==1.37.0 \
  opentelemetry-exporter-otlp-proto-common==1.37.0 \
  opentelemetry-proto==1.37.0 \
  jedi==0.18.2

%pip check
import IPython; IPython.Application.instance().kernel.do_shutdown(True)

### Configure Gemini API

To configure the Gemini API, follow the step-by-step instructions from the `Course Admin` lesson.

But here is a quick check on what you need to run this Notebook:

1.  Get your key from [Google AI Studio](https://aistudio.google.com/app/apikey).

2. In Google Colab, go to the "Secrets" tab (or the key icon) on the left-hand panel.

3. Click "Add new secret" and create a new secret with the following details:

    - Name: GOOGLE_API_KEY 

    - Value: Paste your API key here.

4. Make sure to enable the option "Notebook access".

Now, the code below will load the key from your Colab secrets:

In [ ]:
from utils import env

env.load(required_env_vars=["GOOGLE_API_KEY"])

Trying to load environment variables from `/Users/omar/Documents/ai_repos/course-ai-agents/.env`
Environment variables loaded successfully.


### Import Key Packages

In [ ]:
from enum import Enum
from typing import Callable

from google import genai
from google.genai import types
from pydantic import BaseModel, Field

from utils import pretty_print

### Initialize the Gemini Client

In [ ]:
client = genai.Client()

### Define Constants

We will use the `gemini-2.5-flash` model, which is fast and cost-effective:

In [ ]:
MODEL_ID = "gemini-2.5-flash"

## 2. Tools Definition

Let's implement our mock search tool that will serve as the external knowledge source for our agent. This simplified version focuses on the ReAct mechanics rather than real API integration:

In [ ]:
def search(query: str) -> str:
    """Search for information about a specific topic or query.

    Args:
        query (str): The search query or topic to look up.
    """
    query_lower = query.lower()

    # Predefined responses for demonstration
    if all(word in query_lower for word in ["capital", "france"]):
        return "Paris is the capital of France and is known for the Eiffel Tower."
    elif "react" in query_lower:
        return "The ReAct (Reasoning and Acting) framework enables LLMs to solve complex tasks by interleaving thought generation, action execution, and observation processing."

    # Generic response for unhandled queries
    return f"Information about '{query}' was not found."

We maintain a mapping from tool name to tool function (the tool registry). This lets the model plan with symbolic tool names, while our code safely resolves those names to actual Python functions to execute.

In [ ]:
TOOL_REGISTRY: dict[str, Callable[..., str]] = {
    search.__name__: search,
}

## 3. ReAct Thought Phase

Now let's implement the thought generation phase. This component analyzes the current situation and determines what the agent should do next, potentially suggesting using tools.

First, we prepare the prompt for the thinking part. We implement a function that converts the `TOOL_REGISTRY` to a string XML representation of it, which we insert into the prompt. This way, the LLM knows which tools available and can reason around them.

In [ ]:
def build_tools_xml_description(tool_registry: dict[str, Callable[..., str]]) -> str:
    """Build a minimal XML description of tools using only their docstrings."""
    lines = []
    for tool_name, fn in tool_registry.items():
        doc = (fn.__doc__ or "").strip()
        lines.append(f'\t<tool name="{tool_name}">')
        if doc:
            lines.append("\t\t<description>")
            for line in doc.split("\n"):
                lines.append(f"\t\t\t{line}")
            lines.append("\t\t</description>")
        lines.append("\t</tool>")
    return "\n".join(lines)


# Build a string of XML describing the tools
tools_xml = build_tools_xml_description(TOOL_REGISTRY)

PROMPT_TEMPLATE_THOUGHT = """
You are deciding the next best step for reaching the user goal. You have some tools available to you.

Available tools:
<tools>
{tools_xml}
</tools>

Conversation so far:
<conversation>
{conversation}
</conversation>

State your next **thought** about what to do next as one short paragraph focused on the next action you intend to take and why.
Avoid repeating the same strategies that didn't work previously. Prefer different approaches.

Remember:
- Return ONLY plain natural language text.
- Do NOT emit JSON, XML, function calls, or code.
""".strip()

Here we `print` the prompt with the tool definitions inside.

In [ ]:
print(PROMPT_TEMPLATE_THOUGHT.format(tools_xml=tools_xml, conversation=""))

You are deciding the next best step for reaching the user goal. You have some tools available to you.

Available tools:
<tools>
	<tool name="search">
		<description>
			Search for information about a specific topic or query.
			
			Args:
			    query (str): The search query or topic to look up.
		</description>
	</tool>
</tools>

Conversation so far:
<conversation>

</conversation>

State your next **thought** about what to do next as one short paragraph focused on the next action you intend to take and why.
Avoid repeating the same strategies that didn't work previously. Prefer different approaches.

Remember:
- Return ONLY plain natural language text.
- Do NOT emit JSON, XML, function calls, or code.


We can now implement the `generate_thought` function, which reasons on the best next action to take according to the conversation history.

In [ ]:
def generate_thought(conversation: str, tool_registry: dict[str, Callable[..., str]]) -> str:
    """Generate a thought as plain text (no structured output)."""
    tools_xml: str = build_tools_xml_description(tool_registry)
    prompt: str = PROMPT_TEMPLATE_THOUGHT.format(tools_xml=tools_xml, conversation=conversation)

    response: types.GenerateContentResponse = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt,
    )
    return response.text.strip()

## 4. ReAct Action Phase

Next, let's implement the action phase using function calling. This component determines whether to use a tool or provide a final answer.

In [ ]:
PROMPT_TEMPLATE_ACTION = """
You are selecting the best next action to reach the user goal.

Conversation so far:
<conversation>
{conversation}
</conversation>

Respond either with a tool call (with arguments) or a final answer, but only if you can confidently conclude.
""".strip()

# Dedicated prompt used when we must force a final answer
PROMPT_TEMPLATE_ACTION_FORCED = """
You must now provide a final answer to the user.

Conversation so far:
<conversation>
{conversation}
</conversation>

Provide a concise final answer that best addresses the user's goal.
""".strip()


class ToolCallRequest(BaseModel):
    """A request to call a tool with its name and arguments."""

    tool_name: str = Field(description="The name of the tool to call.")
    arguments: dict = Field(description="The arguments to pass to the tool.")


class FinalAnswer(BaseModel):
    """A final answer to present to the user when no further action is needed."""

    text: str = Field(description="The final answer text to present to the user.")


def generate_action(
    conversation: str, tool_registry: dict[str, Callable[..., str]] | None = None, force_final: bool = False
) -> ToolCallRequest | FinalAnswer:
    """Generate an action by passing tools to the LLM and parsing function calls or final text.

    When force_final is True or no tools are provided, the model is instructed to produce a final answer
    and tool calls are disabled.
    """
    # Use a dedicated prompt when forcing a final answer or no tools are provided
    if force_final or not tool_registry:
        prompt: str = PROMPT_TEMPLATE_ACTION_FORCED.format(conversation=conversation)
        response = client.models.generate_content(model=MODEL_ID, contents=prompt)
        return FinalAnswer(text=response.text.strip())

    # Default action prompt
    prompt = PROMPT_TEMPLATE_ACTION.format(conversation=conversation)

    # Provide the available tools to the model; disable auto-calling so we can parse and run it ourselves
    tools: list[Callable[..., str]] = list(tool_registry.values())
    config = types.GenerateContentConfig(
        tools=tools, automatic_function_calling=types.AutomaticFunctionCallingConfig(disable=True)
    )
    response: types.GenerateContentResponse = client.models.generate_content(
        model=MODEL_ID, contents=prompt, config=config
    )

    # From the reponse, we parse each "part" and check if it's a function call
    candidate = response.candidates[0]
    for part in candidate.content.parts:
        if getattr(part, "function_call", None):
            name = part.function_call.name
            args = dict(part.function_call.args or {})
            return ToolCallRequest(tool_name=name, arguments=args)

    # Otherwise, it's a final answer
    final_answer = "".join(part.text for part in candidate.content.parts)
    return FinalAnswer(text=final_answer.strip())

Why we provide an option to force the final answer? In a ReAct loop we sometimes need to terminate cleanly after a budget of turns (e.g., to avoid infinite loops or excessive tool calls). The force flag lets us ask the model to conclude with a final answer even if, under normal conditions, it might keep calling tools. This ensures graceful shutdown and a usable output at the end of the loop.

Note: In the Action phase we do not inline tool descriptions into the prompt (unlike the Thought phase). Instead, we pass the available Python tool functions through the `tools` parameter to `generate_content`. The client automatically parses these tools and incorporates their definitions/arguments into the model's prompt context, enabling function calling without duplicating tool specs in our prompt text.

## 5. ReAct Observation Phase

This is the third main component of the ReAct loop. In this step, we take the ToolCallRequest created by the generate_action function, run the tool, and return the output.

In [ ]:
def observe(action_request: ToolCallRequest, tool_registry: dict[str, Callable[..., str]]) -> str:
    """
    Execute the selected tool and return the observation text
    (either a result or an error message)
    """
    name = action_request.tool_name
    args = action_request.arguments

    if name not in tool_registry:
        return f"Unknown tool '{name}'. Available: {', '.join(tool_registry)}"

    try:
        return tool_registry[name](**args)
    except Exception as e:
        return f"Error executing tool '{name}': {e}"

In [ ]:
req = ToolCallRequest(tool_name="search", arguments={"query": "capital of France"})
print(observe(req, TOOL_REGISTRY))

Paris is the capital of France and is known for the Eiffel Tower.


## 6. ReAct Control Loop

Now we build the main ReAct control loop that orchestrates the Thought → Action → Observation cycle end-to-end. We treat the conversation between the user and the agent as a sequence of messages. Each message is a step in the dialogue, and each step corresponds to one ReAct unit: it can be a user message, an internal thought, a tool request, the tool's observation, or the final answer.

We'll start by defining the data structures for these messages.

In [ ]:
class MessageRole(str, Enum):
    """Enumeration for the different roles a message can have."""

    USER = "user"
    THOUGHT = "thought"
    TOOL_REQUEST = "tool request"
    OBSERVATION = "observation"
    FINAL_ANSWER = "final answer"


class Message(BaseModel):
    """A message with a role and content, used for all message types."""

    role: MessageRole = Field(description="The role of the message in the ReAct loop.")
    content: str = Field(description="The textual content of the message.")

    def __str__(self) -> str:
        """Provides a user-friendly string representation of the message."""
        return f"{self.role.value.capitalize()}: {self.content}"

We also add a small printer that uses our `pretty_print` module to render each message nicely in the notebook. This makes it easy to follow how the agent alternates between Thought, Action (tool call), and Observation across turns.

In [ ]:
def pretty_print_message(
    message: Message,
    turn: int,
    max_turns: int,
    header_color: str = pretty_print.Color.YELLOW,
    is_forced_final_answer: bool = False,
) -> None:
    if not is_forced_final_answer:
        title = f"{message.role.value.capitalize()} (Turn {turn}/{max_turns}):"
    else:
        title = f"{message.role.value.capitalize()} (Forced):"

    pretty_print.wrapped(
        text=message.content,
        title=title,
        header_color=header_color,
    )

We now use a `Scratchpad` class that wraps a list of `Message` objects and provides `append(..., verbose=False)` to both store and (optionally) pretty-print messages with role-based colors. The scratchpad is serialized each turn so the model can plan the next step.

In [ ]:
class Scratchpad:
    """Container for ReAct messages with optional pretty-print on append."""

    def __init__(self, max_turns: int) -> None:
        self.messages: list[Message] = []
        self.max_turns: int = max_turns
        self.current_turn: int = 1

    def set_turn(self, turn: int) -> None:
        self.current_turn = turn

    def append(self, message: Message, verbose: bool = False, is_forced_final_answer: bool = False) -> None:
        self.messages.append(message)
        if verbose:
            role_to_color = {
                MessageRole.USER: pretty_print.Color.RESET,
                MessageRole.THOUGHT: pretty_print.Color.ORANGE,
                MessageRole.TOOL_REQUEST: pretty_print.Color.GREEN,
                MessageRole.OBSERVATION: pretty_print.Color.YELLOW,
                MessageRole.FINAL_ANSWER: pretty_print.Color.CYAN,
            }
            header_color = role_to_color.get(message.role, pretty_print.Color.YELLOW)
            pretty_print_message(
                message=message,
                turn=self.current_turn,
                max_turns=self.max_turns,
                header_color=header_color,
                is_forced_final_answer=is_forced_final_answer,
            )

    def to_string(self) -> str:
        return "\n".join(str(m) for m in self.messages)

We can now implement the control loop.
- On the first turn, we add the user question.
- Then, at each turn: (1) we get a Thought from the model; (2) we get an Action. If the action is a `FinalAnswer`, we stop. If it's a `ToolCallRequest`, we execute the tool and append the resulting `Observation`, then continue. If we reach the maximum number of turns, we run the action selector one last time with a flag that forces a final answer (no tool calls).

In [ ]:
def react_agent_loop(
    initial_question: str, tool_registry: dict[str, Callable[..., str]], max_turns: int = 5, verbose: bool = False
) -> str | None:
    """
    Implements the main ReAct (Thought -> Action -> Observation) control loop.
    Uses a unified message class for the scratchpad.
    """
    scratchpad = Scratchpad(max_turns=max_turns)

    # Add the user's question to the scratchpad
    user_message = Message(role=MessageRole.USER, content=initial_question)
    scratchpad.append(user_message, verbose=verbose)

    for turn in range(1, max_turns + 1):
        scratchpad.set_turn(turn)

        # Generate a thought based on the current scratchpad
        thought_content = generate_thought(
            scratchpad.to_string(),
            tool_registry,
        )
        thought_message = Message(role=MessageRole.THOUGHT, content=thought_content)
        scratchpad.append(thought_message, verbose=verbose)

        # Generate an action based on the current scratchpad
        action_result = generate_action(
            scratchpad.to_string(),
            tool_registry=tool_registry,
        )

        # If the model produced a final answer, return it
        if isinstance(action_result, FinalAnswer):
            final_answer = action_result.text
            final_message = Message(role=MessageRole.FINAL_ANSWER, content=final_answer)
            scratchpad.append(final_message, verbose=verbose)
            return final_answer

        # Otherwise, it is a tool request
        if isinstance(action_result, ToolCallRequest):
            # Log the tool request
            params_str = ", ".join(f"{k}={repr(v)}" for k, v in action_result.arguments.items())
            scratchpad.append(
                Message(role=MessageRole.TOOL_REQUEST, content=f"{action_result.tool_name}({params_str})"),
                verbose=verbose,
            )

            # Execute and capture the observation (pure function)
            observation_content = observe(action_result, tool_registry)

            # Log the observation
            scratchpad.append(
                Message(role=MessageRole.OBSERVATION, content=observation_content),
                verbose=verbose,
            )

        # Check if the maximum number of turns has been reached. If so, force the action selector to produce a final answer
        if turn == max_turns:
            forced_action = generate_action(
                scratchpad.to_string(),
                force_final=True,
            )
            if isinstance(forced_action, FinalAnswer):
                final_answer = forced_action.text
            else:
                final_answer = "Unable to produce a final answer within the allotted turns."
            final_message = Message(role=MessageRole.FINAL_ANSWER, content=final_answer)
            scratchpad.append(final_message, verbose=verbose, is_forced_final_answer=True)
            return final_answer

Let's test our ReAct agent with a simple factual question that requires a search:

In [ ]:
# A straightforward question requiring a search.
question = "What is the capital of France?"
final_answer = react_agent_loop(question, TOOL_REGISTRY, max_turns=2, verbose=True)

----------------------------------------- User (Turn 1/2): -----------------------------------------
  What is the capital of France?
----------------------------------------------------------------------------------------------------
--------------------------------------- Thought (Turn 1/2): ---------------------------------------
  I should use the `search` tool to find the capital of France, as the user is asking a factual question that can be answered by searching for information.
----------------------------------------------------------------------------------------------------
------------------------------------- Tool request (Turn 1/2): -------------------------------------
  search(query='capital of France')
----------------------------------------------------------------------------------------------------
------------------------------------- Observation (Turn 1/2): -------------------------------------
  Paris is the capital of France and is known for the Eiffel Tower.
--

Last, let's test it with a question that our mock search tool doesn't have knowledge about:

In [ ]:
# A question about a concept the mock search tool doesn't know.
question = "What is the capital of Italy?"
final_answer = react_agent_loop(question, TOOL_REGISTRY, max_turns=2, verbose=True)

----------------------------------------- User (Turn 1/2): -----------------------------------------
  What is the capital of Italy?
----------------------------------------------------------------------------------------------------
--------------------------------------- Thought (Turn 1/2): ---------------------------------------
  The user is asking a factual question about the capital of Italy, which can be directly answered using the search tool. I will use the search tool to find this information.
----------------------------------------------------------------------------------------------------
------------------------------------- Tool request (Turn 1/2): -------------------------------------
  search(query='capital of Italy')
----------------------------------------------------------------------------------------------------
------------------------------------- Observation (Turn 1/2): -------------------------------------
  Information about 'capital of Italy' was not found.

Notice how the ReAct agent tried different strategies to find an answer for the user query, demonstrating live adaptation.

## 7. ReAct with a reasoning model (use built‑in “thinking”)

Here, we set a thinking budget, and define two helpers. The first helper extracts any thought summary text the API returns. 

The second finds the first function call in a response when the model decides to use a tool.

In [ ]:
THINKING_CONFIG = types.ThinkingConfig(
    include_thoughts=True,  # human-readable summaries for transparency/debugging
    thinking_budget=1024,  # tune for latency vs. depth; -1 lets the model decide
)


def extract_thought_summary(response: types.GenerateContentResponse) -> str | None:
    """Collect human-readable thought summaries if present."""
    parts = getattr(response.candidates[0].content, "parts", []) or []
    chunks = [p.text for p in parts if getattr(p, "thought", False) and getattr(p, "text", None)]
    return "\n".join(chunks).strip() if chunks else None


def extract_first_function_call(response: types.GenerateContentResponse):
    """Return (name, args) for the first function call, or None if the model produced a final answer."""
    if getattr(response, "function_calls", None):
        fc = response.function_calls[0]
        return fc.name, dict(fc.args or {})
    parts = getattr(response.candidates[0].content, "parts", []) or []
    for p in parts:
        if getattr(p, "function_call", None):
            return p.function_call.name, dict(p.function_call.args or {})
    return None

Here, we build the request configuration. We provide the Python functions as tools and enable built-in thinking. Automatic function calling is disabled, so we can log each step and run tools ourselves with the `observe` function. 

In [ ]:
def build_config_with_tools(tools: list[Callable[..., str]]) -> types.GenerateContentConfig:
    return types.GenerateContentConfig(
        tools=tools,
        thinking_config=THINKING_CONFIG,
        # We disable the automatic execution of tools, we will use the observe function to run them instead.
        automatic_function_calling=types.AutomaticFunctionCallingConfig(disable=True),
    )

The following is the alternative ReAct loop. The conversation is maintained as a list of `types.Content`. 

After each model turn, we append `response.candidates[0].content` back into `contents` to preserve thought signatures. 

When the model calls a tool, we execute it, log the observation, and then append a `function_response` part so the model can use that observation on the next turn. 

For the visible trace, we keep using a `Scratchpad` object (which we call `human_log`) and our `pretty_print_message` helper function.

In [ ]:
def react_agent_loop_thinking(
    initial_question: str,
    tool_registry: dict[str, Callable[..., str]],
    max_turns: int = 5,
    verbose: bool = True,
) -> str:
    """
    ReAct loop that relies on model-native reasoning:
      - optional thought summaries for visibility,
      - thought signatures preserved by appending model Content back into `contents`,
      - pretty-printed trace using Lesson 8's Scratchpad utilities.
    """

    human_log = Scratchpad(max_turns=max_turns)
    human_log.append(Message(role=MessageRole.USER, content=initial_question), verbose=verbose)

    # Structured "contents" conversation for thought signatures
    contents: list[types.Content] = [types.Content(role="user", parts=[types.Part(text=initial_question)])]
    tools = list(tool_registry.values())
    config = build_config_with_tools(tools)

    for turn in range(1, max_turns + 1):
        human_log.set_turn(turn)

        response = client.models.generate_content(
            model=MODEL_ID,
            contents=contents,
            config=config,
        )

        # 1) Thought summary (if any) — log as your THOUGHT message
        thoughts = extract_thought_summary(response)
        if thoughts:
            human_log.append(Message(role=MessageRole.THOUGHT, content=thoughts), verbose=verbose)

        # 2) Function/Tool call?
        fc = extract_first_function_call(response)
        if fc:
            name, args = fc

            # We keep the model's full response content to preserve the thought signatures
            contents.append(response.candidates[0].content)

            # Log the tool request
            params_str = ", ".join(f"{k}={repr(v)}" for k, v in args.items())
            human_log.append(
                Message(role=MessageRole.TOOL_REQUEST, content=f"{name}({params_str})"),
                verbose=verbose,
            )

            # Execute the tool
            action_request = ToolCallRequest(tool_name=name, arguments=args)
            observation = observe(action_request, tool_registry)

            # Log observation
            human_log.append(Message(role=MessageRole.OBSERVATION, content=observation), verbose=verbose)

            # Send the function response back (standard function-calling protocol)
            fn_resp = types.Part.from_function_response(
                name=name,
                response={"result": observation},
            )
            contents.append(types.Content(role="user", parts=[fn_resp]))
            continue  # next turn

        # 3) No function call => final text
        final_text = (response.text or "").strip()
        human_log.append(Message(role=MessageRole.FINAL_ANSWER, content=final_text), verbose=verbose)
        return final_text

    # 4) Forced finish if we hit max turns: disable tool-calling for the last shot
    forced_config = types.GenerateContentConfig(
        thinking_config=THINKING_CONFIG,
        tool_config=types.ToolConfig(
            function_calling_config=types.FunctionCallingConfig(mode=types.FunctionCallingConfigMode.NONE)
        ),
    )
    forced_response = client.models.generate_content(model=MODEL_ID, contents=contents, config=forced_config)
    final_text = (forced_response.text or "Unable to produce a final answer within the allotted turns.").strip()
    human_log.append(
        Message(role=MessageRole.FINAL_ANSWER, content=final_text),
        verbose=verbose,
        is_forced_final_answer=True,
    )
    return final_text

Now let’s test this new loop using the same questions we used earlier. 

In [ ]:
question = "What is the capital of France?"
final_answer = react_agent_loop_thinking(question, TOOL_REGISTRY, max_turns=3, verbose=True)

----------------------------------------- User (Turn 1/3): -----------------------------------------
  What is the capital of France?
----------------------------------------------------------------------------------------------------
--------------------------------------- Thought (Turn 1/3): ---------------------------------------
  **Let's Find the Capital of France**

Okay, I've got a straightforward, factual question here: "What is the capital of France?"  This is prime territory for a quick search.  Since I need a direct, verifiable answer, the `default_api.search` tool is definitely the way to go.  To ensure the search yields the most accurate results, I'll structure the query precisely as "capital of France". That should provide the necessary information efficiently.
----------------------------------------------------------------------------------------------------
------------------------------------- Tool request (Turn 1/3): -------------------------------------
  search(que

We get the same answer, but now the Thought comes from the summarized version of the model’s internal thinking. Notice how “verbose” these thought summaries are by default. 

Now let’s ask our agent the second question.

In [24]:
question = "What is the capital of Italy?"
final_answer = react_agent_loop_thinking(question, TOOL_REGISTRY, max_turns=3, verbose=True)

----------------------------------------- User (Turn 1/3): -----------------------------------------
  What is the capital of Italy?
----------------------------------------------------------------------------------------------------
--------------------------------------- Thought (Turn 1/3): ---------------------------------------
  **Navigating a Simple Inquiry**

Okay, so I see a straightforward factual question: what's the capital of Italy?  Knowing my way around things, I immediately recognize this as something easily solved with a quick search.  Instead of racking my brain for a fact I should already know, I'll leverage the `search` tool. My internal process is now to formulate the precise query: "capital of Italy."  The `search` tool should be able to instantly provide the answer, and I can then offer it to the user.  Simple and efficient.
----------------------------------------------------------------------------------------------------
------------------------------------- To

--------------------------------------- Thought (Turn 3/3): ---------------------------------------
  **Troubleshooting a Search Tool's Failure**

Okay, so I'm running into a serious issue here. I tried a couple of simple queries, "capital of Italy" and "Italy," expecting straightforward factual answers. However, the search tool is returning "Information about '[query]' was not found" for both. This strongly indicates a problem with the tool itself – it's either malfunctioning, facing some sort of technical glitch, or perhaps its knowledge base is severely limited.

Given my expertise, I know I can't conjure up this information out of thin air. Without a functional search tool, I'm essentially hamstrung when it comes to answering direct factual questions. Therefore, I need to be upfront with the user. I'll have to explain that I'm unable to provide the requested information because the tool I'm relying on isn't working as expected. This will prevent any misunderstanding and allow me to

We also get the same answer as our classic ReAct agent. 